In [2]:

import pandas as pd

from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import os
import chromedriver_autoinstaller
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from concurrent.futures import ThreadPoolExecutor

opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")
opt.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

# opt.add_argument("--lang=en")

# chromedriver_autoinstaller.install()

In [9]:
input_file="./etc/sample.xlsx"
main_df=pd.read_excel(input_file)
main_df

,Club Name,Year,Link
0,Laxachusetts,2017,https://www.insidelacrosse.com/recruiting/club...
1,Laxachusetts,2018,https://www.insidelacrosse.com/recruiting/club...
2,ICON,2017,https://www.insidelacrosse.com/recruiting/club...
3,ICON,2018,https://www.insidelacrosse.com/recruiting/club...
4,ICON,2019,https://www.insidelacrosse.com/recruiting/club...


In [7]:

team_gender="boys"
output_folder_name=f"clubs_teams_{team_gender}"
output_folder = os.path.join('../data/insidelacrosse/',output_folder_name)
if (not os.path.exists(output_folder)):
    os.mkdir(output_folder)
    print('folder created: '+str(output_folder))


In [10]:
def scrape_each_team_players(team_class_list):
    """ scrape each team players """
    driver = webdriver.Chrome(options=opt)
    driver.get(team_class_list[-1])
    df_player=pd.DataFrame()
    try:
        wait_for__player_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, './/table[@class="table box table-striped"]'))
        )
        sleep(1)
        
        player_table=driver.find_element(by=By.XPATH, value='.//table[@class="table box table-striped"]')
        player_table
        table_html = player_table.get_attribute('outerHTML')

        # Convert HTML table to DataFrame
        df_player = pd.read_html(table_html)[0]
        rows = player_table.find_elements(By.XPATH, ".//tbody/tr")

        # Initialize an empty list to store the counts of full stars
        full_star_counts = []

        # Loop through each row
        for row in rows:
            # Find the last column in each row (rating column)
            rating_column = row.find_element(By.XPATH, ".//td[last()]")
            # Count the number of full stars (icon-star-full elements)
            full_star_count = rating_column.find_elements(By.XPATH, ".//i[@class='icon-star-full']")
            # Append the count to the list
            full_star_counts.append(len(full_star_count))
        df_player['Rating'] = full_star_counts
        
        
        df_player["team_name"]=team_class_list[0]
        df_player["year"]=team_class_list[1]
        
        
    except:
#         no table found
        pass
    driver.close()

    return df_player

In [11]:
driver = webdriver.Chrome(options=opt)

In [12]:
for index , row in main_df.iterrows():
    base_link=row["Link"]
    driver.get(base_link)
    print(f"scrapping for {row['Club Name']}     { str(row['Year'])}   at {str(index+1)} index")
    try:
        
        wait_for_tables = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, './/table[@class="table table-striped box"]'))
        )
        sleep(1)

# all tables (boys and girls)
        results=driver.find_elements(by=By.XPATH, value='.//table[@class="table table-striped box"]')
        
        for each_table in results:
            header = each_table.find_element(By.XPATH, './/tr[1]/th[1]').text
        #         find specific gender table 
            if team_gender.lower()  in header.lower():
#                 get all team names, its links and class

                team_names_cells=each_table.find_elements(By.XPATH, './/tr/td[1]')
                team_names = [team.text for team in team_names_cells]
                team_class_cells=each_table.find_elements(By.XPATH, './/tr/td[2]')
                team_classes = [class_.text for class_ in team_class_cells]
                hrefs = each_table.find_elements(By.XPATH, './/tr/td[1]//a[@href]')
                # Extract href attributes and store them in a list of teams 
                href_values = [href.get_attribute('href') for href in hrefs]
                merged_list = list(zip(team_names, team_classes, href_values))
                list_of_lists_teams_n_class = [list(t) for t in merged_list]
                
#                 scrape for each team in parallel

                print(f"scrapping for each team in parallel for   {str(len(list_of_lists_teams_n_class))} teams")


                with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust max_workers as needed
                    # Use executor.map to apply the function to each URL in parallel
                    result_dfs = executor.map(scrape_each_team_players, list_of_lists_teams_n_class)
                concatenated_df = pd.concat(result_dfs, ignore_index=True)
                
                
                concatenated_df.to_csv(f'{output_folder}/{row["Club Name"].replace(" ","_")}_{str(row["Year"])}.csv')
                
                
    except:
        print("no teams found")




scrapping for Laxachusetts     2017   at 1 index
scrapping for each team in parallel for   2 teams
no teams found
scrapping for Laxachusetts     2018   at 2 index
scrapping for each team in parallel for   3 teams
no teams found
scrapping for ICON     2017   at 3 index
scrapping for each team in parallel for   1 teams
no teams found
scrapping for ICON     2018   at 4 index
no teams found
scrapping for ICON     2019   at 5 index
scrapping for each team in parallel for   2 teams
no teams found


In [14]:
concatenated_df

,Rank,Player,Position,High School,Commitment,Rating,team_name,year
0,--,Andrew Rooney,D,Chaminade High School,Siena,0,ICON 2021,2021
1,--,Nick Cowen,D,H Frank Carey High School,Delaware,3,ICON 2021,2021
2,--,TJ Sheehan,A,South Side High School,Denver,0,ICON 2021,2021
3,--,Tommy Casey,M,Chaminade High School,Marquette,0,ICON 2021,2021
4,--,John Jude Considine,A,South Side High School,Hobart,0,ICON 2021,2021
5,--,Thomas Kelly,M,South Side High School,Binghamton,0,ICON 2021,2021


In [15]:
concatenated_df.to_csv(f'{output_folder}/{row["Club Name"].replace(" ","_")}_{str(row["Year"])}.csv')


In [13]:
print("completed....")
driver.close()

completed....
